In [1]:
import cugraph
import cudf
from cugraph.datasets import email_Eu_core

el = email_Eu_core.get_edgelist(download=True)

G = cugraph.Graph(directed=True)
G.from_cudf_edgelist(el, source="src", destination="dst")

seeds = G.select_random_vertices(62, int(0.0001 * len(el)))

In [2]:
sampling_results, offsets, renumber_map = cugraph.uniform_neighbor_sample(
    G,
    seeds,
    [5,5],
    with_replacement=False,
    with_edge_properties=True,
    with_batch_ids=False,
    deduplicate_sources=True,
    prior_sources_behavior='exclude',
    renumber=True,
    return_offsets=True,
    random_state=62,
    use_legacy_names=False,
    compress_per_hop=False,
    compression='CSR',
    include_hop_column=False,
)

/home/nfs/abarghi/cugraph6/python/cugraph/cugraph/sampling/uniform_neighbor_sample.py:244: FutureWarning: The with_edge_properties flag is deprecated and will be removed in the next release.
  warnings.warn(warning_msg, FutureWarning)


In [3]:
sampling_results

,majors,minors,weight,edge_id,edge_type
0,0,0,<NA>,<NA>,<NA>
1,0,1,<NA>,<NA>,<NA>
2,0,2,<NA>,<NA>,<NA>
3,0,3,<NA>,<NA>,<NA>
4,0,4,<NA>,<NA>,<NA>
5,1,0,<NA>,<NA>,<NA>
6,4,0,<NA>,<NA>,<NA>
7,2,1,<NA>,<NA>,<NA>
8,1,4,<NA>,<NA>,<NA>
9,4,4,<NA>,<NA>,<NA>


In [4]:
offsets

,offsets,batch_id
0,0,0
1,5,<NA>
2,25,<NA>


In [5]:
renumber_map

,map,renumber_map_offsets
0,49,0
1,71,20
2,83,<NA>
3,87,<NA>
4,612,<NA>
5,50,<NA>
6,105,<NA>
7,152,<NA>
8,166,<NA>
9,255,<NA>


In [6]:
import cupy
major_offsets = sampling_results['major_offsets'].dropna().values
majors = cudf.Series(cupy.arange(len(major_offsets) - 1))
majors = majors.repeat(cupy.diff(major_offsets))
majors.values
#majors = sampling_results['majors']
majors.values

KeyError: 'major_offsets'

In [ ]:
minors = sampling_results['minors'].dropna()
minors.values

In [ ]:
majors = renumber_map.map.iloc[majors]
majors.values

In [ ]:
minors = renumber_map.map.iloc[minors]
minors.values

In [ ]:
cudf.DataFrame({
    'majors':majors.values,
    'minors':minors.values
})

In [ ]:
seeds

In [ ]:
el[el.src==50]

In [ ]:
for i in range(len(majors)):
    print(i, len(el[(el.src==majors.iloc[i]) & (el.dst==minors.iloc[i])]))